# PROJET #2 - Using earlier time zone stock index information to predict stock market direction

## 1. The premise

The premise is straightforward: financial markets are increasingly global, and if you follow the sun from Asia to Europe to the US and so on, **information from an earlier time zone can be used to your advantage in a later time zone**.

The following table shows a number of stock market indices from around the globe, their closing times in Eastern Standard Time (EST), and the delay in hours between the close of that index and the close of the **S&P/TSX Composite Index** in Toronto. This makes EST the base time zone. For example, Australian markets close for the day 15 hours before the Toronto Stock Exchange close. If the close of the All Ords in Australia is a useful predictor of the close of the S&P/TSX Composite Index for a given day we can use that information to guide our trading activity. Continuing our example of the Australian All Ords, if this index closes up and we think that means the S&P/TSX Composite Index will close up as well then we should either buy stocks that compose the S&P/TSX Composite Index or, more likely, an ETF that tracks the S&P/TSX Composite Index. In reality, the situation is more complex because there are commissions and tax to account for. But as a first approximation, we'll assume an index closing up indicates a gain, and vice-versa.

|Index|Country|Closing Time (EST)|Hours Before S&P/TSX CI Close|source|
|---|---|---|---|---|
|[All Ords](https://en.wikipedia.org/wiki/All_Ordinaries)|Australia|0100|15|[yahoo ^AORD](https://ca.finance.yahoo.com/quote/%5EAORD?p=%5EAORD)|
|[Nikkei 225](https://en.wikipedia.org/wiki/Nikkei_225)|Japan|0200|14|[yahoo ^N225](https://finance.yahoo.com/quote/%5EN225/)|
|[Hang Seng](https://en.wikipedia.org/wiki/Hang_Seng_Index)|Hong Kong|0400|12|[yahoo ^HSI](https://finance.yahoo.com/quote/%5EHSI/)|
|[DAX](https://en.wikipedia.org/wiki/DAX)|Germany|1130|4.5|[yahoo ^GDAXI](https://finance.yahoo.com/quote/%5EGDAXI/)|
|[FTSE 100](https://en.wikipedia.org/wiki/FTSE_100_Index)|UK|1130|4.5|[LSE](https://www.londonstockexchange.com/statistics/ftse/ftse.htm)|
|[NYSE Composite](https://en.wikipedia.org/wiki/NYSE_Composite)|US|1600|0|[yahoo ^NYA](https://www.londonstockexchange.com/statistics/ftse/ftse.htm)|
|[Dow Jones Industrial Average](https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average)|US|1600|0|[yahoo ^DJI](https://finance.yahoo.com/quote/%5EDJI/)|
|[S&P 500](https://en.wikipedia.org/wiki/S%26P_500_Index)|US|1600|0|[yahoo ^GSPC](https://ca.finance.yahoo.com/quote/%5EGSPC/)|
|[S&P/TSX Composite Index](https://en.wikipedia.org/wiki/Toronto_Stock_Exchange)|Canada|1600|0|[yahoo ^GSPTSE](https://ca.finance.yahoo.com/quote/%5EGSPTSE?p=^GSPTSE)|

## 2. Set up

First, install and import necessary libraries.

In [ ]:
!pip install --upgrade pip
!pip install pandas
!pip install xlrd
!pip install scipy
!pip install sklearn
!pip install seaborn
!pip install openpyxl

In [ ]:
import numpy as np
import pandas as pd
from pandas.plotting import autocorrelation_plot
from pandas.plotting import scatter_matrix

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(20,15)})

In [ ]:
import warnings
warnings.simplefilter('ignore')

## 3. Get the data

The data covers roughly 5 years, using the date range from 31/08/2014 to 37/07/2019. Data comes from the S&P/TSX Composite Index (SNPTSX), S&P 500 (S&P), NYSE, Dow Jones Industrial Average (DJIA), Nikkei 225 (Nikkei), Hang Seng, FTSE 100 (FTSE), DAX, and All Ordinaries (AORD) indices.

#### CSV files

In [ ]:
snp = pd.read_csv('^GSPC.csv', index_col='Date', usecols=['Date','Close'], parse_dates=True)
nyse = pd.read_csv('^NYA.csv', index_col='Date', usecols=['Date','Close'], parse_dates=True)
djia = pd.read_csv('^DJI.csv', index_col='Date', usecols=['Date','Close'], parse_dates=True)
nikkei = pd.read_csv('^N225.csv', index_col='Date', usecols=['Date','Close'], parse_dates=True)
hangseng = pd.read_csv('^HSI.csv', index_col='Date', usecols=['Date','Close'], parse_dates=True)
dax = pd.read_csv('^GDAXI.csv', index_col='Date', usecols=['Date','Close'], parse_dates=True)
aord = pd.read_csv('^AORD.csv', index_col='Date', usecols=['Date','Close'], parse_dates=True)
snptsx = pd.read_csv('^GSPTSE.csv', index_col='Date', usecols=['Date','Close'], parse_dates=True)

#### Excel files

In [ ]:
ftse = pd.read_excel('FTSE.xlsx', index_col=0, skiprows=16, usecols=['Date','FTSE 100']).sort_index()

## 4. Munge the data

In the first instance, munging the data is straightforward. The closing prices are of interest, so for convenience extract the closing prices for each of the indices into a single Pandas DataFrame, called closing_data. **Because not all of the indices have the same number of values, mainly due to bank holidays, gaps will be forward-filled**. This means that, if a value isn't available for day N, fill it with the value for another day, such as N-1 or N-2, so that it contains the latest available value.

In [ ]:
dates = pd.date_range(start='2014-08-31', end='2019-07-31')
closing_data = pd.DataFrame(index=dates)

In [ ]:
closing_data = closing_data.join(snp[['Close']].rename(columns={'Close' : 'snp_close'}))
closing_data = closing_data.join(nyse[['Close']].rename(columns={'Close' : 'nyse_close'}))
closing_data = closing_data.join(djia[['Close']].rename(columns={'Close' : 'djia_close'}))
closing_data = closing_data.join(nikkei[['Close']].rename(columns={'Close' : 'nikkei_close'}))
closing_data = closing_data.join(hangseng[['Close']].rename(columns={'Close' : 'hangseng_close'}))
closing_data = closing_data.join(dax[['Close']].rename(columns={'Close' : 'dax_close'}))
closing_data = closing_data.join(aord[['Close']].rename(columns={'Close' : 'aord_close'}))
closing_data = closing_data.join(ftse[['FTSE 100']].rename(columns={'FTSE 100' : 'ftse_close'}))
closing_data = closing_data.join(snptsx[['Close']].rename(columns={'Close' : 'snptsx_close'}))

**All rows having a nan values are dropped. Imputation will be tried in a second research**

In [ ]:
closing_data = closing_data.dropna()

At this point, five years of time series for eight financial indices have been sourced, combined the pertinent data into a single data structure, and harmonized the data to have the same number of entries.

In [ ]:
del snp, nyse, djia, nikkei, hangseng, dax, aord, ftse, snptsx

In [ ]:
closing_data.head()

## 5. Exploratory Data Analysis (EDA)

Exploratory Data Analysis (EDA) is foundational to working with machine learning, and any other sort of analysis. EDA means getting to know your data, getting your fingers dirty with your data, feeling it and seeing it. The end result is you know your data very well, so when you build models you build them based on an actual, practical, physical understanding of the data, not assumptions or vaguely held notions. You can still make assumptions of course, but EDA means you will understand your assumptions and why you're making those assumptions. 

First, take a look at the data.

In [ ]:
closing_data.describe()

You can see that the various indices operate on scales differing by orders of magnitude. It's best to scale the data so that, for example, operations involving multiple indices aren't unduly influenced by a single, massive index.

Plot the data.
N.B. A super-useful trick-ette is to assign the return value of plot to _ so that you don't get text printed before the plot itself.

In [ ]:
_ = plt.plot(closing_data)
_ = plt.legend(closing_data.columns, ncol=2, loc='upper left');

As expected, the structure isn't uniformly visible for the indices. Divide each value in an individual index by the maximum value for that index., and then replot. The maximum value of all indices will be 1.

In [ ]:
closing_data_scaled = closing_data / closing_data.max(axis=0)

In [ ]:
_ = plt.plot(closing_data_scaled)
_ = plt.legend(closing_data_scaled.columns, ncol=2, loc='upper left');

You can see that, over the five-year period, these indices are correlated. Notice that sudden drops from economic events happened globally to all indices, and they otherwise exhibited general rises. This is an good start, though not the complete story. Next, plot autocorrelations for each of the indices. The autocorrelations determine correlations between current values of the index and lagged values of the same index. The goal is to determine whether the lagged values are reliable indicators of the current values. If they are, then we've identified a correlation.

In [ ]:
_ = autocorrelation_plot(closing_data['snp_close'], label='snp_close')
_ = autocorrelation_plot(closing_data['nyse_close'], label='nyse_close')
_ = autocorrelation_plot(closing_data['djia_close'], label='djia_close')
_ = autocorrelation_plot(closing_data['nikkei_close'], label='nikkei_close')
_ = autocorrelation_plot(closing_data['hangseng_close'], label='hangseng_close')
_ = autocorrelation_plot(closing_data['ftse_close'], label='ftse_close')
_ = autocorrelation_plot(closing_data['dax_close'], label='dax_close')
_ = autocorrelation_plot(closing_data['aord_close'], label='aord_close')


_ = plt.legend(loc='upper right')

You should see strong autocorrelations, positive for around 300 lagged days, then going negative. This tells us something we should intuitively know: if an index is rising it tends to carry on rising, and vice-versa. It should be encouraging that what we see here conforms to what we know about financial markets.

Next, look at a scatter matrix, showing everything plotted against everything, to see how indices are correlated with each other.

In [ ]:
_ = scatter_matrix(closing_data_scaled, diagonal='kde')

You can see significant correlations across the board, further evidence that the premise is workable and one market can be influenced by another. 

As an aside, this process of gradual, incremental experimentation and progress is the best approach and what you probably do normally. With a little patience, we'll get to some deeper understanding.

The actual value of an index is not that useful for modeling. It can be a useful indicator, but to get to the heart of the matter, we need a time series that is stationary in the mean, thus having no trend in the data. There are various ways of doing that, but they all essentially look at the difference between values, rather than the absolute value. In the case of market data, the usual practice is to work with logged returns, calculated as the natural logarithm of the index today divided by the index yesterday:

$$ln(V_t/V_{t-1})$$


There are more reasons why the log return is preferable to the percent return (for example the log is normally distributed and additive), but they don't matter much for this work. What matters is to get to a stationary time series.

Calculate and plot the log returns in a new DataFrame.

In [ ]:
log_return_data = np.log(closing_data / closing_data.shift())
log_return_data.columns = ['snp_log_return','nyse_log_return','djia_log_return','nikkei_log_return','hangseng_log_return','dax_log_return','aord_log_return','ftse_log_return','snptsx_log_return']

log_return_data.describe()

Looking at the log returns, you should see that the mean, min, max are all similar. You could go further and center the series on zero, scale them, and normalize the standard deviation, but there's no need to do that at this point. Let's move forward with plotting the data, and iterate if necessary.

In [ ]:
_ = plt.plot(log_return_data)
_ = plt.legend(log_return_data.columns, loc='upper right')

You can see from the plot that the log returns of our indices are similarly scaled and centered, with no visible trend in the data. It's looking good, so now look at autocorrelations.

In [ ]:
_ = autocorrelation_plot(log_return_data['snp_log_return'], label='snp_log_return')
_ = autocorrelation_plot(log_return_data['nyse_log_return'], label='nyse_log_return')
_ = autocorrelation_plot(log_return_data['djia_log_return'], label='djia_log_return')
_ = autocorrelation_plot(log_return_data['nikkei_log_return'], label='nikkei_log_return')
_ = autocorrelation_plot(log_return_data['hangseng_log_return'], label='hangseng_log_return')
_ = autocorrelation_plot(log_return_data['ftse_log_return'], label='ftse_log_return')
_ = autocorrelation_plot(log_return_data['dax_log_return'], label='dax_log_return')
_ = autocorrelation_plot(log_return_data['aord_log_return'], label='aord_log_return')
_ = autocorrelation_plot(log_return_data['snptsx_log_return'], label='snptsx_log_return')

_ = plt.legend(loc='upper right')

No autocorrelations are visible in the plot, which is what we're looking for. Individual financial markets are Markov processes, knowledge of history doesn't allow you to predict the future. 

You now have time series for the indices, stationary in the mean, similarly centered and scaled. That's great! Now start to look for signals to try to predict the close of the S&P/TSX Composite Index. 

Look at a scatterplot to see how the log return indices correlate with each other.

In [ ]:
_ = scatter_matrix(log_return_data, figsize=(20, 20), diagonal='kde')

The story with the previous scatter plot for log returns is more subtle and more interesting. The US indices are strongly correlated, as expected. The other indices, less so, which is also expected. But there is structure and signal there. Now let's move forward and start to quantify it so we can start to choose features for our model.

First look at how the log returns for the closing value of the S&P/TSX Composite Index correlate with the closing values of other indices available on the same day. This essentially means to assume the indices that close before the S&P/TSX Composite Index are available and the others are not.

In [ ]:
tmp = pd.DataFrame()
tmp['snptsx_1'] = log_return_data['snptsx_log_return']
tmp['snp_1'] = log_return_data['snp_log_return']
tmp['nyse_1'] = log_return_data['nyse_log_return'].shift()
tmp['djia_1'] = log_return_data['djia_log_return'].shift()
tmp['ftse_0'] = log_return_data['ftse_log_return']
tmp['dax_0'] = log_return_data['dax_log_return']
tmp['hangseng_0'] = log_return_data['hangseng_log_return']
tmp['nikkei_0'] = log_return_data['nikkei_log_return']
tmp['aord_0'] = log_return_data['aord_log_return']
tmp.corr().iloc[:,0]

Here, we are directly working with the premise. We're correlating the close of the S&P/TSX Composite Index with signals available before the close of the S&P/TSX Composite Index.  And you can see that the S&P/TSX Composite Index close is correlated with European indices at around 0.5 for the FTSE and DAX, which is a strong correlation, and Asian/Oceanian indices at around 0.23-0.26, which is a significant correlation, but not with US indices. We have available signals from other indices and regions for our model.

Now look at how the log returns for the S&P closing values correlate with index values from the previous day to see if they previous closing is predictive. Following from the premise that financial markets are Markov processes, there should be little or no value in historical values.

In [ ]:
tmp = pd.DataFrame()
tmp['snptsx_0'] = log_return_data['snptsx_log_return']
tmp['snp_1'] = log_return_data['snp_log_return'].shift(2)
tmp['nyse_1'] = log_return_data['nyse_log_return'].shift(2)
tmp['djia_1'] = log_return_data['djia_log_return'].shift(2)
tmp['ftse_0'] = log_return_data['ftse_log_return'].shift(1)
tmp['dax_0'] = log_return_data['dax_log_return'].shift(1)
tmp['hangseng_0'] = log_return_data['hangseng_log_return'].shift(1)
tmp['nikkei_0'] = log_return_data['nikkei_log_return'].shift(1)
tmp['aord_0'] = log_return_data['aord_log_return'].shift(1)
tmp.corr().iloc[:,0]

You should see little to no correlation in this data, meaning that yesterday's values are no practical help in predicting today's close. Let's go one step further and look at correlations between today and the the day before yesterday.

In [ ]:
tmp = pd.DataFrame()
tmp['snptsx_0'] = log_return_data['snptsx_log_return']
tmp['snp_1'] = log_return_data['snp_log_return'].shift(3)
tmp['nyse_1'] = log_return_data['nyse_log_return'].shift(3)
tmp['djia_1'] = log_return_data['djia_log_return'].shift(3)
tmp['ftse_0'] = log_return_data['ftse_log_return'].shift(2)
tmp['dax_0'] = log_return_data['dax_log_return'].shift(2)
tmp['hangseng_0'] = log_return_data['hangseng_log_return'].shift(2)
tmp['nikkei_0'] = log_return_data['nikkei_log_return'].shift(2)
tmp['aord_0'] = log_return_data['aord_log_return'].shift(2)

tmp.corr().iloc[:,0]

Again, there are little to no correlations.

In [ ]:
del tmp, closing_data, closing_data_scaled

### Summing up the EDA

At this point, you've done a good enough job of exploratory data analysis. You've visualized our data and come to know it better. You've transformed it into a form that is useful for modelling, log returns, and looked at how indices relate to each other. You've seen that indices from Europe strongly correlate with US indices, and that indices from Asia/Oceania significantly correlate with those same indices for a given day. You've also seen that if you look at historical values, they do not correlate with today's values. Summing up:

* European indices from the same day were a strong predictor for the S&P/TSX Composite Index close.
* Asian/Oceanian indices from the same day were a significant predictor for the S&P/TSX Composite Index close.
* Indices from previous days were not good predictors for the S&P/TSX Composite Index close.

## 6. Feature Engineering

At this point, we can see a model:

* We'll predict whether the S&P/TSX Composite Index close today will be higher or lower than yesterday.
* We'll use all our data sources: NYSE, DJIA, Nikkei, Hang Seng, FTSE, DAX, AORD.
* We'll use three sets of data points—T, T-1, and T-2—where we take the data available on day T or T-n, meaning today's non-US data and yesterday's US data.

Predicting whether the log return of the S&P/TSX Composite Index is positive or negative is a classification problem. That is, we want to choose one option from a finite set of options, in this case positive or negative. This is the base case of classification where we have only two values to choose from, known as binary classification, or logistic regression.

This uses the findings from of our exploratory data analysis, namely that log returns from other regions on a given day are strongly correlated with the log return of the S&P/TSX Composite Index, and there are stronger correlations from those regions that are geographically closer with respect to time zones. However, our models also use data outside of those findings. For example, we use data from the past few days in addition to today.  There are two reasons for using this additional data. First, we're adding additional features to our model for the purpose of this solution to see how things perform. which is not a good reason to add features outside of a tutorial setting. Second, machine learning models are very good at finding weak signals from data.

From a training and testing perspective, time series data is easy. Training data should come from events that happened before test data events, and be contiguous in time.  Otherwise,  your model would be trained on events from "the future", at least as compared to the test data. It would then likely perform badly in practice, because you can’t really have access to data from the future. That means random sampling or cross validation don't apply to time series data. Decide on a training-versus-testing split, and divide your data into training and test datasets.

In this case, you'll create the features together with the following response column(s):

*  snptsx_log_return_positive, which is 1 if the log return of the S&P/TSX close is positive, and 0 otherwise. 

In [ ]:
log_return_data['snptsx_log_return_positive'] = 0
log_return_data.loc[log_return_data['snptsx_log_return'] >= 0, 'snptsx_log_return_positive'] = 1

In [ ]:
training_test_data = pd.DataFrame(
  columns=[
    'snptsx_log_return_positive',
    'snptsx_log_return_1','snptsx_log_return_2','snptsx_log_return_3',
    'snp_log_return_1', 'snp_log_return_2', 'snp_log_return_3',
    'nyse_log_return_1', 'nyse_log_return_2', 'nyse_log_return_3',
    'djia_log_return_1', 'djia_log_return_2', 'djia_log_return_3',
    'nikkei_log_return_0', 'nikkei_log_return_1', 'nikkei_log_return_2',
    'hangseng_log_return_0', 'hangseng_log_return_1', 'hangseng_log_return_2',
    'ftse_log_return_0', 'ftse_log_return_1', 'ftse_log_return_2',
    'dax_log_return_0', 'dax_log_return_1', 'dax_log_return_2',
    'aord_log_return_0', 'aord_log_return_1', 'aord_log_return_2'])

for i in range(7, len(log_return_data)):   
    training_test_data = training_test_data.append(
        {'snptsx_log_return_positive': log_return_data['snptsx_log_return_positive'].iloc[i],
        'snptsx_log_return_1':log_return_data['snptsx_log_return'].iloc[i-1],
        'snptsx_log_return_2':log_return_data['snptsx_log_return'].iloc[i-2],
        'snptsx_log_return_3':log_return_data['snptsx_log_return'].iloc[i-3],
        'snp_log_return_1':log_return_data['snp_log_return'].iloc[i-1],
        'snp_log_return_2':log_return_data['snp_log_return'].iloc[i-2],
        'snp_log_return_3':log_return_data['snp_log_return'].iloc[i-3],
        'nyse_log_return_1':log_return_data['nyse_log_return'].iloc[i-1],
        'nyse_log_return_2':log_return_data['nyse_log_return'].iloc[i-2],
        'nyse_log_return_3':log_return_data['nyse_log_return'].iloc[i-3],
        'djia_log_return_1':log_return_data['djia_log_return'].iloc[i-1],
        'djia_log_return_2':log_return_data['djia_log_return'].iloc[i-2],
        'djia_log_return_3':log_return_data['djia_log_return'].iloc[i-3],
        'nikkei_log_return_0':log_return_data['nikkei_log_return'].iloc[i],
        'nikkei_log_return_1':log_return_data['nikkei_log_return'].iloc[i-1],
        'nikkei_log_return_2':log_return_data['nikkei_log_return'].iloc[i-2],
        'hangseng_log_return_0':log_return_data['hangseng_log_return'].iloc[i],
        'hangseng_log_return_1':log_return_data['hangseng_log_return'].iloc[i-1],
        'hangseng_log_return_2':log_return_data['hangseng_log_return'].iloc[i-2],
        'ftse_log_return_0':log_return_data['ftse_log_return'].iloc[i],
        'ftse_log_return_1':log_return_data['ftse_log_return'].iloc[i-1],
        'ftse_log_return_2':log_return_data['ftse_log_return'].iloc[i-2],
        'dax_log_return_0':log_return_data['dax_log_return'].iloc[i],
        'dax_log_return_1':log_return_data['dax_log_return'].iloc[i-1],
        'dax_log_return_2':log_return_data['dax_log_return'].iloc[i-2],
        'aord_log_return_0':log_return_data['aord_log_return'].iloc[i],
        'aord_log_return_1':log_return_data['aord_log_return'].iloc[i-1],
        'aord_log_return_2':log_return_data['aord_log_return'].iloc[i-2]},
        ignore_index=True)

In [ ]:
training_test_data.describe()

In [ ]:
training_test_data.columns

## 7. Modelisation

<h1 style="color: red">À partir de cette cellule, c'est à vous de jouer !</h1>